In [ ]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.eda_methods import show_barplot, load_data, load_data_all
from pandas.plotting import register_matplotlib_converters

pd.set_option('display.max_colwidth', 125)
register_matplotlib_converters()

In [ ]:
# IMPORTANT - Figure out how to correctly convert old hashes to new hashes

# project_name = 'zeppelin'
# try:
#     tmp =\
#         pd.read_csv(
#             'assets/data/{0}/new_{0}_commits.csv'.format(project_name),
#             nrows=None,
#             header=None
#         )
# except UnicodeDecodeError:
#     tmp =\
#         pd.read_csv(
#             'assets/data/{0}/new_{0}_commits.csv'.format(project_name),
#             nrows=None,
#             header=None,
#             encoding='utf-16'
#         )
# tmp = tmp.set_index(0).apply(pd.to_datetime, axis=1)
# tmp['dates_match'] = tmp.apply(lambda x: x.duplicated().sum(), axis=1).astype('bool')

# # finds
# grouped = tmp.groupby(2)
# g_index =\
#     [
#             x for x in grouped[1].apply(
#                 lambda x: pd.to_datetime(x[0], utc=True) 
#                 if (x.count() > 1).sum() else ''
#             ).values if x
#     ]
# tmp = tmp.reset_index().set_index(1)
# tmp.index = pd.to_datetime(tmp.index, utc=True, infer_datetime_format=True)

# transforms date column into datetime_index
# new_commits =\
#     pd.Series(
#         tmp[0].values,
#         index=pd.to_datetime(
#                 tmp[1].values,
#                 infer_datetime_format=True,
#                 utc=True
#             )
#     )
# new_commits.name = project_name

In [ ]:
old, new, cc, bic = load_data_all()

In [ ]:
# shows duplicated bug introducing commits (BIC) for each project
print('Total and duplicated bic for each project:\n\n\t\t\t(total, duplicated)')
bic.groupby(level=0).apply(lambda x: (x.count(), x.duplicated().sum()))

In [ ]:
# removes duplicated BICs
print('Total BICs after removal of duplicates:\n')

project_bic = bic.groupby(level=0)

# saves unique BICs
ubic = project_bic.apply(lambda x: x.drop_duplicates(keep='first')).droplevel(0, axis=0)

print(ubic.groupby(level=0, axis=0).count())
del bic

In [ ]:
'''
shows all conflicting 'created_at' commits
proceed with manual investigation of how to solve this (HUGE ISSUE!!!)
'''
# old.droplevel(0, axis=0)[old.droplevel(0, axis=0).index.duplicated(keep=False)]
# new.droplevel(0, axis=0)[new.droplevel(0, axis=0).index.duplicated(keep=False)]

In [ ]:
# new.reset_index().iloc[old.reset_index().index][0].values
'''
prints projects with different number of commits
'''
new_g = new.groupby(level=0)
old_g = old.groupby(level=0)

for ng, og in zip(new_g, old_g):
    # ng[1] = ng[1][ng[1].index.isin(og[1].index)]
    if len(ng[1])-1 != len(og[1]):
        print(ng[0], (len(ng[1]) - len(og[1]))-1)

In [ ]:
# new.reset_index().iloc[old.reset_index().index][0].values
'''
ensures that only commits present in 'old' are considered
'''
new_g = new.groupby(level=0)
old_g = old.groupby(level=0)
tmp = []

for ng, og in zip(new_g, old_g):
    tmp.append(ng[1][ng[1].index.isin(og[1].index)])

new = pd.concat(tmp)
del tmp
# del new_g, old_g

In [ ]:
# aggregates old and new commits to facilitate conversion
convert = pd.DataFrame()

convert['old_hash'] = old
convert['new_hash'] =\
    new.reset_index()\
    .iloc[old.reset_index()\
    .index][0].values

# converts bic commits to their equivalent in 'new'
ubic =\
    convert.set_index('old_hash')\
    .squeeze()[ubic.values]\
    .dropna()\
    .reset_index(drop=True)

del convert

In [ ]:
# # creates a dataframe to support conversion of old hashes into new ones
# conv_df = pd.DataFrame()

# conv_df['old_hash'] = old.sort_index()
# conv_df['new_hash'] =\
#     new.sort_index()\
#     .reset_index(drop=True)[
#         old.sort_index()\
#         .reset_index(drop=True)\
#         .index
#     ].values